In [1]:
! apt-get update && apt-get install -y libsndfile1 ffmpeg
! pip install Cython
! pip install packaging
! pip -q install nemo_toolkit['asr'] Cython packaging
! pip install pandas torch

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Get:2 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1225 B]
Get:3 https://packages.cloud.google.com/apt cloud-sdk InRelease [1616 B]       
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:6 https://packages.cloud.google.com/apt google-fast-socket InRelease [1071 B]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Get:8 https://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [24.1 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1672 kB]
Hit:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease              
Get:11 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [3089 kB]
Get:12 https://packages.cloud.google.com/apt cloud-sdk/main all Packages

In [1]:
import os
import torch
import pandas as pd
from nemo.collections.asr.models import EncDecCTCModel

[NeMo W 2024-07-07 20:42:32 nemo_logging:393] /home/amr-khaled/anaconda3/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    


In [2]:
model_path = "best_model_checkpoint/quartznet_15x5_final.nemo"


# To Load the model
quartznet = EncDecCTCModel.restore_from(model_path)

if torch.cuda.is_available():
    print('Cuda')
    quartznet.to('cuda')

[NeMo W 2024-07-07 20:42:46 nemo_logging:393] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /kaggle/input/full-manifest/full_manifest.json
    sample_rate: 16000
    labels:
    - ' '
    - أ
    - ا
    - آ
    - إ
    - ب
    - ت
    - ث
    - ج
    - ح
    - خ
    - د
    - ذ
    - ر
    - ز
    - س
    - ش
    - ص
    - ض
    - ط
    - ظ
    - ع
    - غ
    - ف
    - ق
    - ك
    - ل
    - م
    - ن
    - ه
    - و
    - ي
    - ى
    - ئ
    - ﻻ
    - ﻹ
    - ﻷ
    - ﻵ
    - ة
    - ء
    - ؤ
    batch_size: 32
    trim_silence: true
    max_duration: 15
    shuffle: true
    num_workers: 8
    pin_memory: true
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    
[NeMo W 2024-07-07 20:42:46 nemo_logging:393] If you inte

[NeMo I 2024-07-07 20:42:46 nemo_logging:381] PADDING: 16
[NeMo I 2024-07-07 20:42:47 nemo_logging:381] Model EncDecCTCModel was successfully restored from /media/amr-khaled/Data/me/tempd/7asebatiya_7elwan/best_model_checkpoint/quartznet_15x5_final.nemo.
Cuda


In [4]:
def transcribe_audio_files(folder_path, output_csv):
    # List to store file paths
    file_paths = []

    # Loop through all files in the specified directory
    for filename in os.listdir(folder_path):
        if filename.endswith(".wav"):  # Check if the file is a WAV file
            file_path = os.path.join(folder_path, filename)
            file_paths.append(file_path)

    # Transcribe all audio files at once
    with torch.no_grad():
        transcriptions = quartznet.transcribe(file_paths)

    # List to store transcription results
    results = []

    # Loop through transcriptions and file paths
    for filename, transcription in zip(os.listdir(folder_path), transcriptions):
        if filename.endswith(".wav"):  # Check if the file is a WAV file
            # Remove the .wav extension from the filename
            audio_name = filename.replace('.wav', '')
            results.append({
                'audio': audio_name,
                'transcript': transcription
            })

    # Convert results to a DataFrame and save to CSV
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)

Transcribing:   0%|          | 0/432 [00:00<?, ?it/s]

In [ ]:
folder_path = "path/to/audio_folder"
output_csv = "transcriptions.csv"
transcribe_audio_files(folder_path, output_csv)